In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/ 

import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Flatten, Dense

In [ ]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_KERNEL_SIZE = hp.HParam('kernel_size', hp.Discrete([3,5]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.01,0.001]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_KERNEL_SIZE, HP_BATCH_SIZE, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
def train_test_model(hparams):
  input = keras.Input(shape=(32, 32, 3))
  x = Conv2D(hparams[HP_NUM_UNITS], (hparams[HP_KERNEL_SIZE], hparams[HP_KERNEL_SIZE]), activation='relu', padding='same')(input)
  x = BatchNormalization()(x)
  x = Conv2D(hparams[HP_NUM_UNITS], (hparams[HP_KERNEL_SIZE], hparams[HP_KERNEL_SIZE]), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  
  x = Flatten()(x)
  x = Dropout(hparams[HP_DROPOUT])(x)
  
  x = Dense(1024, activation='relu')(x)
  x = Dropout(hparams[HP_DROPOUT])(x)
  
  output = Dense(10, activation='softmax')(x)

  model = keras.Model(input, output, name="cifar_model")  

  if hparams[HP_OPTIMIZER] == 'adam':
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate= hparams[HP_LEARNING_RATE], beta_1=0.9, beta_2 = 0.999),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
  else:
    model.compile(
        optimizer= tf.keras.optimizers.SGD(learning_rate= hparams[HP_LEARNING_RATE]),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )

  model.fit(x_train, y_train, batch_size = hparams[HP_BATCH_SIZE], epochs=10) # Run with few epoch to speed things up
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      for kernel_size in HP_KERNEL_SIZE.domain.values:
        for batch_size in HP_BATCH_SIZE.domain.values:  
          for learning_rate in HP_LEARNING_RATE.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
                HP_KERNEL_SIZE: kernel_size,
                HP_BATCH_SIZE: batch_size,
                HP_LEARNING_RATE: learning_rate,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam', 'kernel_size': 3, 'batch_size': 32, 'learning_rate': 0.001}
Epoch 1/10
1563/1563 [==============================] - 26s 16ms/step - loss: 1.2691 - accuracy: 0.5638
Epoch 2/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.8087 - accuracy: 0.7189
Epoch 3/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.6479 - accuracy: 0.7754
Epoch 4/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.5406 - accuracy: 0.8145
Epoch 5/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.4408 - accuracy: 0.8463
Epoch 6/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.3556 - accuracy: 0.8762
Epoch 7/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.2817 - accuracy: 0.9021
Epoch 8/10
1563/1563 [==============================] - 23s 15ms/step - loss: 0.2280 - accuracy: 0.9203
Epoch 9/10
1563/1563 [=====

In [ ]:
%tensorboard --logdir logs/hparam_tuning